In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [3]:
import numpy as np

def read_images(path):
    with open(path, "rb") as f:
        data = np.frombuffer(f.read(), dtype=np.uint8, offset=16)
    return data.reshape(-1, 28, 28)

X_train = read_images("Dataset/fashion-mnist/train-images-idx3-ubyte/train-images-idx3-ubyte")
X_test  = read_images("Dataset/fashion-mnist/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte")


In [4]:
def read_labels(path):
    with open(path, "rb") as f:
        return np.frombuffer(f.read(), dtype=np.uint8, offset=8)

y_train = read_labels("Dataset/fashion-mnist/train-labels-idx1-ubyte/train-labels-idx1-ubyte")
y_test  = read_labels("Dataset/fashion-mnist/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte")


## Defining the model
<hr>

### initialization


In [5]:
init_1 = keras.initializers.VarianceScaling(scale=2.0, mode='fan_avg')

<hr>
<hr>

### normalization


####  keras.layers.BatchNormalization()

#### keras.layers.Activation("elu")

##### use_bias=False => because batch-normaization automatically add a shift value 

In [6]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),

    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, kernel_initializer=init_1, use_bias=False),

    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(75, use_bias=False), # use_bias=False => because batch-normaization automatically add a shift value 

    keras.layers.BatchNormalization(),
    keras.layers.LeakyReLU(alpha=0.3),
    keras.layers.Dense(10, activation="softmax")
])

c:\Users\SH.Rayan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
c:\Users\SH.Rayan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 784)            │         3,136 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │        78,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 75)             │         7,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 75)             │           300 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 75)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           760 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 90,496 (353.50 KB)

 Trainable params: 88,578 (346.01 KB)

 Non-trainable params: 1,918 (7.49 KB)

In [8]:
sgd =  keras.optimizers.SGD(clipvalue=1.0)

In [9]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer = sgd,
              metrics = ["accuracy"]
              )

In [10]:
# history = model.fit(X_train,y_train,epochs=10, validation_split=0.2)

In [11]:
# model.evaluate(X_test,y_test)

In [12]:
# model.save("models/model0.h5")

## Transfer-Learning
<hr>

In [13]:
model0 = keras.models.load_model("models/model0.h5")

## Cloning the model
<hr>

In [14]:
model0.get_weights()[4]

array([[ 0.07943863,  0.02270434, -0.10742782, ...,  0.06702015,
         0.07275703,  0.03770913],
       [-0.11193924,  0.04757405, -0.0140844 , ...,  0.06712791,
         0.10389724, -0.03250265],
       [-0.00147658, -0.04133404, -0.00965767, ..., -0.0230976 ,
         0.03769337,  0.10212652],
       ...,
       [-0.00278829,  0.03714105,  0.03467741, ...,  0.09847248,
         0.04988788, -0.03514279],
       [-0.04880447,  0.11505166, -0.01949594, ..., -0.07193203,
         0.12341086, -0.00442115],
       [-0.00649335,  0.11895999, -0.01834931, ...,  0.0406763 ,
         0.03361779, -0.02343226]], shape=(784, 100), dtype=float32)

In [15]:
model0_clone = keras.models.clone_model(model0)
model0_clone.set_weights(model0.get_weights())

In [27]:
model0_clone.get_weights()[4]

array([[ 0.07943863,  0.02270434, -0.10742782, ...,  0.06702015,
         0.07275703,  0.03770913],
       [-0.11193924,  0.04757405, -0.0140844 , ...,  0.06712791,
         0.10389724, -0.03250265],
       [-0.00147658, -0.04133404, -0.00965767, ..., -0.0230976 ,
         0.03769337,  0.10212652],
       ...,
       [-0.00278829,  0.03714105,  0.03467741, ...,  0.09847248,
         0.04988788, -0.03514279],
       [-0.04880447,  0.11505166, -0.01949594, ..., -0.07193203,
         0.12341086, -0.00442115],
       [-0.00649335,  0.11895999, -0.01834931, ...,  0.0406763 ,
         0.03361779, -0.02343226]], shape=(784, 100), dtype=float32)

In [26]:
#  after fittin model 1
model1.get_weights()[4]

array([[ 0.07943863,  0.02270434, -0.10742782, ...,  0.06702015,
         0.07275703,  0.03770913],
       [-0.11193924,  0.04757405, -0.0140844 , ...,  0.06712791,
         0.10389724, -0.03250265],
       [-0.00147658, -0.04133404, -0.00965767, ..., -0.0230976 ,
         0.03769337,  0.10212652],
       ...,
       [-0.00278829,  0.03714105,  0.03467741, ...,  0.09847248,
         0.04988788, -0.03514279],
       [-0.04880447,  0.11505166, -0.01949594, ..., -0.07193203,
         0.12341086, -0.00442115],
       [-0.00649335,  0.11895999, -0.01834931, ...,  0.0406763 ,
         0.03361779, -0.02343226]], shape=(784, 100), dtype=float32)

<hr>

In [18]:
model0.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 784)            │         3,136 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │        78,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 75)             │         7,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 75)             │           300 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 75)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           760 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 90,498 (353.51 KB)

 Trainable params: 88,578 (346.01 KB)

 Non-trainable params: 1,918 (7.49 KB)

 Optimizer params: 2 (12.00 B)

In [19]:
model1 = keras.models.Sequential(model0.layers[:-1])
model1.add(keras.layers.Dense(1,activation="sigmoid"))
model1.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 784)            │         3,136 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │        78,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 75)             │         7,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 75)             │           300 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 75)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 89,736 (350.53 KB)

 Trainable params: 87,818 (343.04 KB)

 Non-trainable params: 1,918 (7.49 KB)

### Freezing the model

In [20]:
for layer in model1.layers[:-1]:
    layer.trainable =False
    print(layer.trainable)




False
False
False
False
False
False
False
False


In [21]:
model1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 784)            │         3,136 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │        78,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 100)            │           400 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 75)             │         7,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 75)             │           300 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 75)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 89,736 (350.53 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 89,736 (350.53 KB)

In [22]:
X_train_new = X_train[:5000,:]
y_train_new = np.where(y_train == 9,1,0 )[:5000]



In [23]:
model1.compile(loss="binary_crossentropy",optimizer="sgd",metrics=["accuracy"])




In [25]:
model1.fit(X_train_new,y_train_new,epochs=15,validation_split=0.15)

Epoch 1/15
133/133 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9012 - loss: 0.2986 - val_accuracy: 0.9547 - val_loss: 0.1811
Epoch 2/15
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9649 - loss: 0.1403 - val_accuracy: 0.9760 - val_loss: 0.1213
Epoch 3/15
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9711 - loss: 0.1053 - val_accuracy: 0.9800 - val_loss: 0.0978
Epoch 4/15
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9760 - loss: 0.0886 - val_accuracy: 0.9800 - val_loss: 0.0845
Epoch 5/15
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9779 - loss: 0.0786 - val_accuracy: 0.9813 - val_loss: 0.0758
Epoch 6/15
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9795 - loss: 0.0720 - val_accuracy: 0.9827 - val_loss: 0.0699
Epoch 7/15
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9807 - loss: 0.0671 - val_accuracy: 0.9827 - val_loss: 0.0653
Epoch 8/15
133/133 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9816 - loss: 0.0635 - val_accuracy: 0.